# Download training dataset

- Note that the training data is pre-processed by N'Sarm.
- Since we, team Optimizer-6, have discussed with all minor teams in House-Optimizer and we agreed with size problem of text from each bounding box.
 - We tried to check the image size, considered from the distribution of width and height dimensions.
 - We found that most of images, contain 100-200 pixels for width and heights. 
- The brief idea is normalizing all images, cropping from bounding box, to keep features related to image sizes.
 - The solution is placing the cropped image onto new background with same size, 105x105 components in this case.
 - This size is not magic number but we decided to use them because the chosen model architecture.
 - From team discussion, we firstly focused on DeepFont model which requires 105x105 pixels for the input images. 
- All images are processed by resize based on original dimensions
 - if old_height <= 105 and old_width <= 105:
 - elif old_height <= 105 and old_width > 105:
 - elif old_height > 105 and old_width <= 105:
 - else:
- Then, they are processed as follows:
 - converts to grayscale
 - GaussianBlur
 - adaptiveThreshold
 - binarized by threshold_sauvola


In [ ]:
# zip file from Optimizer-5 (Sauvola)
!gdown --id 1mBr6_Q_cvqPgYMlzCLK7dB202wTsLjnQ&export=download

In [ ]:
!unzip -q /content/Thresh_Sauvola.zip -d /content/

## Split train data into train and validation data folder

In [ ]:
import os
import glob
from sklearn.model_selection import train_test_split
import shutil
import csv

import numpy as np
import matplotlib.pyplot as plt

from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

In [ ]:
MAIN_PATH = '/content/Thresh_Sauvola'
files = os.listdir("/content/Thresh_Sauvola")

# label for each class
name_label = ['Angsana_New', 'Cordia_New', 'DM_Shining_Star_Regular',  
              'FC_Knomphing_Regular', 'Kunlasatri', 'TH_Chakra_Petch',
              'TH_Charm_of_AU', 'TH_Mali_Grade6', 'TH_Sarabun', 'fonttintin']
size_label = ['12', '14', '16', '18', '20', '22', '24']
style_label = ['italic-bold', 'italic-normal', 'normal-bold', 'normal-normal']


# for font family and font style
for file in files:
  image_path = os.path.join(MAIN_PATH, file)
  
  for label in name_label:
    if not os.path.isdir('Train/' + label):
      os.makedirs('Train/' + label)

    if label in file:
      shutil.move(image_path, 'Train/' + label)

# NOTE: Since font size number, 12_14_16_18_20_22_24, is sometimes found in ParentId 
# for font size
# for file in files:
#   image_path = os.path.join(MAIN_PATH, file)
  
#   for label in size_label:
#     if not os.path.isdir('Train/' + label):
#       os.makedirs('Train/' + label)

#     if label in file.split('-')[1]:
#       shutil.move(image_path, 'Train/' + label)

In [ ]:
# define function for splitting the initial training data into train and validation dataset
def split_data(path_to_data, path_to_save_train, path_to_save_val, split_size=0.3):
    
    folders = os.listdir(path_to_data)
    
    for folder in folders:
        
        full_path = os.path.join(path_to_data, folder)
        images_paths = glob.glob(os.path.join(full_path, '*.jpg'))
        
        x_train, x_val = train_test_split(images_paths, test_size = split_size)
        
        for x in x_train:
            
            path_to_folder = os.path.join(path_to_save_train, folder)
            
            if not os.path.isdir(path_to_folder):
                os.makedirs(path_to_folder)
                
            shutil.copy(x, path_to_folder)
            
        for x in x_val:
            
            path_to_folder = os.path.join(path_to_save_val, folder)
            if not os.path.isdir(path_to_folder):
                os.makedirs(path_to_folder)
                
            shutil.copy(x, path_to_folder)

In [ ]:
# define path for moving data into train and validation folder
PATH_main = "/content"

path_to_data = os.path.join(PATH_main, "Train")
path_to_save_train = os.path.join(PATH_main, "training_data/train")
path_to_save_val = os.path.join(PATH_main, "training_data/val")
    
split_data(path_to_data,  
           path_to_save_train=path_to_save_train,  
           path_to_save_val=path_to_save_val)

# Building a NN the functional way

In [ ]:
# import labraries for NN architecture
import os
import matplotlib.pyplot as plt
import pandas as pd 
import numpy as np
import cv2
import tensorflow as tf
from tensorflow import keras

from keras.layers import Conv2D, Input, Dense, MaxPool2D, BatchNormalization, GlobalAvgPool2D, Flatten, Dropout
from tensorflow.keras.applications.inception_v3 import InceptionV3
from keras.models import Sequential 
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, CSVLogger

In [ ]:
# model architecture: InceptionV3 (transfer learning)
def class_model(nbr_classes):
  
  # Note that the input shape equals to (105,105,3)
  inception = InceptionV3(include_top=False, weights='imagenet', input_shape=(105, 105, 3))
  model = Sequential()
  model.add(inception)
  model.add(Flatten())
  model.add(Dropout(0.4))
  model.add(Dense(100,activation='relu'))
  model.add(Dropout(0.2))
  model.add(Dense(nbr_classes,activation='softmax'))

  return model

In [ ]:
# # model architecture: simple CNN
# def class_model(nbr_classes):

#   my_input = Input(shape=(105, 105, 3))

#   x = Conv2D(32, (3,3), activation='relu')(my_input)
#   x = MaxPool2D()(x)
#   x = BatchNormalization()(x)

#   x = Conv2D(64, (3,3), activation='relu')(x)
#   x = MaxPool2D()(x)
#   x = BatchNormalization()(x)

#   x = Conv2D(128, (3,3), activation='relu')(x)
#   x = MaxPool2D()(x)
#   x = BatchNormalization()(x)

#   x = GlobalAvgPool2D()(x)
#   x = Dense(128, activation='relu')(x)
#   x = Dense(nbr_classes, activation='softmax')(x)

#   return tf.keras.Model(inputs=my_input,  
#                outputs=x)

In [ ]:
# class_model(10).summary()

In [ ]:
from keras.utils.vis_utils import plot_model

# Plot model graph
plot_model(class_model(10), show_shapes=True, show_layer_names=True, to_file='model.png')
from IPython.display import Image
Image(retina=True, filename='model.png')

# Create Datagenerator

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
def create_generators(batch_size, train_data_path, val_data_path):

  train_preprocessor = ImageDataGenerator(
      rescale = 1 / 255.,
      # rotation_range = 10,      # Potential improvements
      # width_shift_range=0.1     # Potential improvements
  )

  test_preprocessor = ImageDataGenerator(
      rescale = 1 / 255.,
  )

  train_generator = train_preprocessor.flow_from_directory(
      train_data_path,  
      class_mode='categorical',
      target_size=(105,105,),
      color_mode='rgb',
      shuffle=True,
      batch_size=batch_size    
  )

  val_generator = test_preprocessor.flow_from_directory(
      val_data_path,  
      class_mode='categorical',
      target_size=(105,105,),
      color_mode='rgb',
      shuffle=False,
      batch_size=batch_size    
  )

  # test_generator = test_preprocessor.flow_from_directory(
  #     test_data_path,  
  #     class_mode='categorical',|
  #     target_size=(105,105,3),
  #     color_mode='rgb',
  #     shuffle=False,
  #     batch_size=batch_size    
  # )

  return train_generator, val_generator

In [ ]:
PATH_main = "/content"

path_to_train = os.path.join(PATH_main, "training_data/train")
path_to_val = os.path.join(PATH_main, "training_data/val")
# path_to_test = os.path.join(PATH_main, "Test")

batch_size = 128

train_generator, val_generator = create_generators(batch_size, path_to_train, path_to_val)

# Compiling the model and fitting the data

In [ ]:
PATH_main = "/content"

# callbacks
path_to_save_model = os.path.join(PATH_main, "Models")

ckpt_saver = ModelCheckpoint(
    path_to_save_model,  
    monitor="val_accuracy",  
    mode="max",  
    save_best_only=True,  
    save_freq='epoch',  
    verbose=1
)

# EarlyStopping
early_stopping = EarlyStopping(monitor="val_accuracy",  
                               patience=10, verbose=1)

# Reduce learning rate
reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.2,patience=2, min_lr=1e-6)

# CSV Log
csv_logger = CSVLogger('training.log', separator=',', append=False)

In [ ]:
nbr_classes = train_generator.num_classes
model = class_model(nbr_classes)
optimizer = tf.keras.optimizers.Adam(learning_rate = 0.001)

model.compile(optimizer=optimizer,  
              loss='categorical_crossentropy',  
              metrics=['accuracy'])

model.fit(train_generator,  
          epochs=50,  
          batch_size=batch_size,  
          validation_data=val_generator,  
          callbacks=[ckpt_saver, early_stopping, reduce_lr, csv_logger])

Saving model

In [ ]:
# !mv Models model_family_inceptionv3_all

In [ ]:
# !zip -r model_family_inceptionv3_all.zip model_family_inceptionv3_all

# Predict test dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

### Load test set

In [ ]:
# zip file from N'Sarm (Sauvola) "TEST_Public"
!gdown --id "1_tl4EXJkiFjJvHljIbCkZsebKlYqXJLP&export=download"

In [ ]:
# zip file from N'Sarm (Sauvola) "TEST_Public_Private"
!gdown --id 1VUwIN1NYcuWmH6h19RBWM1Vm9H_gHckG&export=download

In [ ]:
!unzip -q /content/TestSetFinalFinal.zip -d /content/

In [ ]:
# define path for test dataset
PATH_TEST = '/content/TestSetFinalFinal'
file = os.listdir(PATH_TEST)
print(len(file))
print(file[0])

In [ ]:
# check the file name (image Id)
id = file[0].replace('.jpg', '')
id

In [ ]:
# collect the image pixels

img_list = []
img_id = []
for i in range(len(file)):
    id = file[i].replace('.jpg', '')
    img_id.append(id)
    path_img = os.path.join(PATH_TEST, file[i])
    img = cv2.imread(path_img)
    # img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # img = img[..., tf.newaxis]
    img = img.astype("float32") / 255.
    # img = pad(img) # padding to the same size
    # img_shape = img.shape
    img_list.append(img)
    
    # img_shape_array.append(img_shape)
    # print(img.shape)
img_list = np.array(img_list)

In [ ]:
print(img_list[1].shape)
print(img_id[1])

In [ ]:
img_array = np.asarray(img_list)
print(img_array.shape)

## Load saved model

In [ ]:
# PATH_model_Fam = "/content/drive/MyDrive/font_Models/InceptionV3_all/model_family_inceptionv3_all"
# model_Fam = tf.keras.models.load_model(PATH_model_Fam)

# PATH_model_Size = "/content/drive/MyDrive/font_Models/InceptionV3_all/model_size_inceptionv3_all"
# model_Size = tf.keras.models.load_model(PATH_model_Size)

PATH_model_Style = "/content/drive/MyDrive/font_Models/InceptionV3_all/model_style_inception_all"
model_Style = tf.keras.models.load_model(PATH_model_Style)

In [ ]:
del img_list

In [ ]:
# Load CSVLogger to check the accuracy and loss history
log_data = pd.read_csv('training.log', sep=',', engine='python')

## Predict the test image

In [ ]:
# y_pred_Fam = model_Fam.predict(img_array)
# y_pred_Fam = np.argmax(y_pred_Fam, axis = 1)

# y_pred_Size = model_Size.predict(img_array)
# y_pred_Size = np.argmax(y_pred_Size, axis = 1)

y_pred_Style = model_Style.predict(img_array)
y_pred_Style = np.argmax(y_pred_Style, axis = 1)

In [ ]:
len(y_pred_Style)

## Merge predicted results from all labels

In [ ]:
res_zip = zip(img_id, y_pred_Fam, y_pred_Size, y_pred_Style)
df_res = pd.DataFrame(res_zip, columns = ['Id', '', 'style-weight'])
df_res.head()

## Re-label the encoded results

In [ ]:
df_res = df_res.replace({'name' : {
                            0 : 'Angsana_New',
                            1 : 'Cordia_New',
                            2 : 'DM_Shining_Star_Regular',
                            3: 'FC_Knomphing_Regular',
                            4:'Kunlasatri',
                            5:'TH_Chakra_Petch',
                            6:'TH_Charm_of_AU',
                            7:'TH_Mali_Grade6',
                            8:'TH_Sarabun',
                            9:'fonttintin'},  
                        'size' : {
                                0 : '12px',
                                1 : '14px',
                                2 : '16px',
                                3 : '18px',
                                4 : '20px',
                                5 : '22px',
                                6 : '24px'},
                        'style-weight' : {
                                  0 : 'italic-bold',
                                  1 : 'italic-normal',
                                  2 : 'normal-bold',
                                  3 : 'normal-normal'}
                        })

In [ ]:
df_res.head()

In [ ]:
df_res['file'] = df_res['name'] + '.ttf'
df_res['file'] = df_res['file'].replace('_', ' ', regex=True)
df_res = df_res.reindex(columns=['Id', 'name', 'file', 'size'])
df_res.head()

In [ ]:
df_res.to_csv("csv_temp.csv", index = False)

- If we cannot predict all features together, we will predict each feature, separately.
- Then, we will read csv file for previous predicted results to merge with the latest one.

In [ ]:
df_temp = pd.read_csv("csv_temp.csv")

- Now, we add the latest prediction 

In [ ]:
df_temp["style-weight"] = df_res["style-weight"]

- The entired predictions must be contained in csv form.

In [ ]:
df_try = df_temp

In [ ]:
NAME = pd.DataFrame(list(zip(df_try['Id']+ '_name', df_try['name'])), columns=['Id', 'Predicted'])
FILE = pd.DataFrame(list(zip(df_try['Id']+ '_file', df_try['file'])), columns=['Id','Predicted'])
SIZE = pd.DataFrame(list(zip(df_try['Id']+ '_size', df_try['size'])), columns=['Id','Predicted'])
STYLE = pd.DataFrame(list(zip(df_try['Id']+ '_style-weight', df_try['style-weight'])), columns=['Id','Predicted'])

In [ ]:
print(len(NAME))
print(len(FILE))
print(len(SIZE))
print(len(STYLE))

In [ ]:
df_final = pd.concat([NAME, FILE, SIZE, STYLE], axis=0)

- The sample submission file, imported from Kaggle, is used as referenced index columns.

In [ ]:
df_sub = pd.read_csv("/content/sample_submission.csv")
df_sub.head()

In [ ]:
final_results = pd.merge(df_sub, df_final, on='Id', how="outer")

In [ ]:
final_results.shape

In [ ]:
final_results = final_results.drop(columns=['Predicted_x'])

In [ ]:
final_results.rename(columns={'Predicted_y': 'Predicted'}).to_csv('submission_Inception_Test2.csv', index=False, encoding='utf8')

# Merging two test results from public and private round

In [ ]:
test1 = pd.read_csv("/content/submission_Inception_Test1.csv")  # public
test2 = pd.read_csv("/content/submission_Inception_Test2.csv")  # private

In [ ]:
final_df = pd.merge(test1, test2, on=['Id'], how="outer")

In [ ]:
final_df.to_csv("/content/submission_Inception_Test_Final.csv", index=False)